# Testing NOBIL data

### connecting with duckdb

In [167]:
import duckdb
from pathlib import Path
import pandas as pd

path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.csmd_table_dump;")
    df_csmd = cursor.df()


df_csmd


,id,name,ocpidb_mapping_stasjon_id,street,house_number,zipcode,city,municipality_id,municipality,county_id,...,created,updated,station_status,land_code,international_id,_dlt_load_id,_dlt_id,description_of_location,user_comment,contact_info
0,14671,Agnesfridsvägen 193,SE#VAT#75dbcba6-8bdf-4c70-9b5e-c7e2ab4dd50d,Agnesfridsvägen,193,21375,Malmö,1280,Malmö,12,...,2023-04-24 11:31:42+02:00,2025-09-21 01:13:17+02:00,1,SWE,SWE_14671,1764762727.0329604,QyqHeJdi0WaupA,None,None,None
1,54376,Reload Filipstad,SE#VIR#SEVIRLJX9T0WRY,John Ericssonsgatan,8,68230,Filipstad,1782,Filipstad,17,...,2025-06-13 03:44:32+02:00,2025-06-14 03:44:28+02:00,1,SWE,SWE_54376,1764762727.0329604,UfDz5w594egwaA,None,None,None
2,36573,Källhage Fast AB,SE#CNE#10e955c3-9683-4e7a-de1f-08dc39d02872,Toredalsgatan,3,41871,Göteborg,1480,Göteborg,14,...,2024-11-02 00:00:34+01:00,2025-11-16 20:18:09+01:00,1,SWE,SWE_36573,1764762727.0329604,JLeC7RVTv15ZPg,None,None,None
3,54401,Jönköpings kommun - P-hus Smedjan,SE#VLT#6cbb6162-2630-40ef-a66d-44a91bf9a6ed,Södra Strandgatan,59,55320,Jönköping,0680,Jönköping,06,...,2025-06-13 07:00:56+02:00,2025-06-13 07:00:56+02:00,1,SWE,SWE_54401,1764762727.0329604,4QDrcKZ3/DF79w,None,None,None
4,18856,500,DK#MON#s384017,Flygarvägen,37,17569,Järfälla,0123,Järfälla,01,...,2023-12-15 12:57:07+01:00,2025-11-18 05:22:17+01:00,1,SWE,SWE_18856,1764762727.0329604,dNXMqb/kVKEwmg,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8987,63534,Pay N Charge - Sotenäs GK,None,Önna,,45693,Hunnebostrand,1427,Sotenäs,14,...,2025-08-20 12:22:45+02:00,2025-08-20 14:29:37+02:00,1,SWE,SWE_63534,1764762727.0329604,/4OO/mBrNwwYyQ,Laddstolparna är strategiskt placerade på Sote...,Ingen app eller registrering krävs – laddninge...,Telefon: 010-585 84 00\r\nE-post: info@payncha...
8988,63540,Hedin Supercharge Kristianstad MB/BYD,SE#HED#SEVIRLIPSPSHC1,Östra Blekingevägen,12,29154,Kristianstad,1290,Kristianstad,12,...,2025-08-21 03:41:02+02:00,2025-08-21 03:41:02+02:00,1,SWE,SWE_63540,1764762727.0329604,oqE/wEby6SBRQw,None,None,None
8989,63542,Hedin Supercharge Hedin Group HQ,SE#HED#SEVIRLO4T9JN9W,Betagatan,2,43149,Mölndal,1481,Mölndal,14,...,2025-08-21 03:41:04+02:00,2025-08-21 03:41:04+02:00,1,SWE,SWE_63542,1764762727.0329604,39CtoTMT7/nFoA,None,None,None
8990,63548,LGG - Laddplatser,SE#VLT#7cd9d654-0f5d-4252-9b23-72069c263010,Franska vägen,8,39356,Kalmar,0880,Kalmar,08,...,2025-08-21 07:00:41+02:00,2025-08-21 07:00:41+02:00,1,SWE,SWE_63548,1764762727.0329604,SKewCdFEGItDXA,None,None,None


In [170]:
df_csmd.columns

Index(['id', 'name', 'ocpidb_mapping_stasjon_id', 'street', 'house_number',
       'zipcode', 'city', 'municipality_id', 'municipality', 'county_id',
       'county', 'owned_by', 'operator', 'number_charging_points', 'position',
       'image', 'available_charging_points', 'created', 'updated',
       'station_status', 'land_code', 'international_id', '_dlt_load_id',
       '_dlt_id', 'description_of_location', 'user_comment', 'contact_info'],
      dtype='object')

In [168]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.status_online_table_dump;")
    df_st = cursor.df()
    
df_st

,station_id,attr_key,attrtypeid,attrname,attrvalid,trans,attrval,_dlt_load_id,_dlt_id
0,14671,24,24,Open 24h,1,Yes,1,1764762727.0329604,QjLKB1aHma3EIg
1,14671,21,21,Real-time information,2,No,,1764762727.0329604,eyZSpgVei3070Q
2,14671,22,22,Public funding,4,None,,1764762727.0329604,jVPSUGp6n6VYtg
3,14671,7,7,Parking fee,2,No,False,1764762727.0329604,kNFreqUXGUdYdA
4,14671,6,6,Time limit,2,No,0,1764762727.0329604,wGjvqwTIi9P1qw
...,...,...,...,...,...,...,...,...,...
62939,63601,22,22,Public funding,4,None,,1764762727.0329604,t8HtcTGPjzteQw
62940,63601,7,7,Parking fee,2,No,False,1764762727.0329604,BgHLQjN2ow6FfQ
62941,63601,6,6,Time limit,2,No,0,1764762727.0329604,5TA4nYqBP0sqmA
62942,63601,3,3,Location,1,Street,,1764762727.0329604,8JKf5iGfe8UWig


In [169]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.connector_table_dump;")
    df_conn = cursor.df()
    
df_conn

,station_id,connector_nr,attrtypeid,attrname,attrvalid,trans,attrval,_dlt_load_id,_dlt_id
0,14671,1,1,Accessibility,5,Payment,,1764762727.0329604,DmeOLzZzSBGUGA
1,14671,1,4,Connector,32,Type 2,,1764762727.0329604,ro2pgHTaEMrdnw
2,14671,1,5,Charging capacity,11,22 kW - 400V 3-phase max 32A,,1764762727.0329604,+VyB4oTvD+IQlg
3,14671,1,12,Voltage (V),2,Voltage (V),230,1764762727.0329604,zqeJNZRsyBivdw
4,14671,1,17,Vehicle type,1,Rechargeable cars and vans,,1764762727.0329604,H0ce6AEaq/UZFg
...,...,...,...,...,...,...,...,...,...
709731,63601,2,26,Energy carrier,1,Electricity,,1764762727.0329604,JGKpsmrJ23wLVg
709732,63601,2,27,EVSE UID,7,EVSE UID,00CH2100000000000000000491,1764762727.0329604,VWdvFNs95D8v2Q
709733,63601,2,28,EVSE ID,8,EVSE ID,SE*PAI*EPAI00296*2,1764762727.0329604,fq3tZnH/+M+6dg
709734,63601,2,29,Connector ID,9,Connector ID,00CH2100000000000000000491,1764762727.0329604,yuVCdmq20N1cLQ


In [7]:
from datetime import date
import datetime

x = datetime.datetime.now()

today_date = date(x.year, x.month, x.day).isoformat()

print(today_date)


2025-12-04
